### Handoffs

A way for an agent to invoke another agent

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from agents import Agent, Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    ),
    output_type=Tutorial
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=(
        "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(outline_builder, "Loops in Java")
print(tutorial_response.final_output)

Here's a suggested outline for a tutorial on "Loops in Java":

### Introduction
- Overview of loops and their importance in programming.
- Brief introduction to looping structures in Java.

### Types of Loops in Java
1. **For Loop**
   - Syntax and flow diagram.
   - Use cases and examples.
   - Nested for loops.

2. **While Loop**
   - Syntax and flow diagram.
   - Use cases and examples.
   - Comparison with for loop.

3. **Do-While Loop**
   - Syntax and flow diagram.
   - Key differences from the while loop.
   - Examples and edge cases.

### Enhanced For Loop
- Introduction to the enhanced for loop (for-each).
- Syntax and use cases.
- Examples with arrays and collections.

### Loop Control Statements
1. **Break Statement**
   - Usage within loops.
   - Examples with for, while, and do-while loops.

2. **Continue Statement**
   - How it functions within loops.
   - Practical examples.

3. **Return Statement**
   - Using return to exit loops.
   - Examples in methods with loops.

#

In [3]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [4]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
result.final_output

Handing off to math tutor agent


'Adding two numbers is a straightforward arithmetic operation. Here’s how you add 2 and 2:\n\n1. **Identify the Numbers**: You have the numbers 2 and 2.\n\n2. **Basic Addition**: When you add, you combine their values. So, \\(2 + 2\\).\n\n3. **Calculate the Sum**: Start from 2 and count up two more: \\(2, 3, 4\\).\n\nTherefore, \\(2 + 2 = 4\\).\n\n### Example\nImagine you have 2 apples and you get 2 more apples. In total, you now have 4 apples. \n\nThis simple process can be visualized on a number line if you find it helpful:\n\n- Start at 2.\n- Move 2 steps to the right.\n- You land on 4. \n\nThis is the sum when you add 2 and 2 together.'

In [5]:
result = await Runner.run(triage_agent, "How did WW1 start?")
result.final_output

Handing off to history tutor agent


"World War I, also known as the Great War, began in 1914. Several factors contributed to its outbreak, but the immediate cause was the assassination of Archduke Franz Ferdinand of Austria.\n\n### Key Events Leading to the War:\n\n1. **Assassination of Archduke Franz Ferdinand (June 28, 1914):**\n   - The heir to the Austro-Hungarian throne, Archduke Franz Ferdinand, and his wife, Sophie, were assassinated in Sarajevo by Gavrilo Princip, a Bosnian Serb member of the nationalist group, the Black Hand.\n\n2. **Austria-Hungary's Reaction:**\n   - Austria-Hungary, seeking to punish Serbia for supporting nationalist movements, issued an ultimatum to Serbia with demands that were intentionally unacceptable to ensure a pretext for war.\n\n3. **Alliance System:**\n   - The complex system of alliances contributed to the escalation. Major alliances included the Triple Entente (France, Russia, and the United Kingdom) and the Triple Alliance (Germany, Austria-Hungary, and Italy).\n\n4. **Declaratio

In [6]:
from agents import function_tool

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

result = await Runner.run(customer_service_agent, "Hello how much are tickets?")
print(result.final_output)

Could you please specify which event or service you are inquiring about tickets for? That way, I can provide you with the correct information.


In [7]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

billing_agent = Agent(
    name="Billing agent",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    <Fill in the rest of your prompt here>.""",
)